# Goal for how to organize the signatures

Copy all ribosomal-removed signatures to these subfolders


Using `DNA`, `protein`, and `dayhoff` for shorthand now. In reality would use the full sketch id, e.g. `alphabet-DNA__ksize-21__scaled-10`

```
mouse2mouse/
    self2self-bootstrapped/
        0--train-mouse1-single-cells/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-01/
                ...
        1--test-mouse1-single-cells/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-01/
                ...                
        2--train-mouse1-merged-celltype-sigs/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-01/
                ...
        3--train-mouse1-celltype-remove-common-kmers/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-01/
                ...
        4--train-mouse1--celltype-sbt-dbs/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-01/
                ...
        5--search-results/
            iteration-00/
                DNA/
                protein/
                dayhoff/
            iteration-00/
                ...
    mouse1-mouse2/
        0--train-mouse1-single-cells/
            DNA/
            protein/
            dayhoff/
        1--test-mouse2-single-cells/
            DNA/
            protein/
            dayhoff/
        2--train-mouse1-merged-celltype-sigs/
            DNA/
            protein/
            dayhoff/
        3--train-mouse1-celltype-remove-common-kmers/
            DNA/
            protein/
            dayhoff/
        4--train-mouse1-celltype-sbt-dbs/
            DNA/
            protein/
            dayhoff
        5--search-results/
            DNA/
            protein/
            dayhoff/
train-mouse/
    0--protein-coding-fastas/
        nucleotide/
        peptide/
    1--single-cell-sigs/
        DNA/
        protein/
        dayhoff/
    2--merged-celltype-sigs/
        DNA/
        protein/
        dayhoff/
    3--celltype-remove-common-kmers/
        DNA/
        protein/
        dayhoff/
    4--celltype-sbt-dbs/
        DNA/
        protein/
        dayhoff/
test-human/
    0--protein-coding-fastas/
        nucleotide/
        peptide/
    1--single-cell-sigs/
        DNA/
        protein/
        dayhoff/
    2--mouse-search-results/
        DNA/
        protein/
        dayhoffl
test-lemur/
    0--protein-coding-fastas/
        nucleotide/
        peptide/
    1--single-cell-sigs/
        protein/
        dayhoff/
    2--mouse-search-results/
        DNA/
        protein/
        dayhoff/
test-bat/
    0--protein-coding-fastas/
        nucleotide/
        peptide/
    1--single-cell-sigs/
        protein/
        dayhoff/
    2--mouse-search-results/
        DNA/
        protein/
        dayhoff/
```

# Imports

In [32]:
import glob
import os
import shutil

import pandas as pd
import scanpy as sc
import sig_utils
from joblib import Parallel, delayed
from tqdm import tqdm

import create_sourmash_command_utils as csc_utils

import sig_reorg

pd.options.display.max_rows = 500

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Def Describe

In [2]:
%%file pandas_utils.py

def describe(df, random=False):
    print(df.shape)
    print("--- First 5 entries ---")
    display(df.head())
    if random:
        print('--- Random subset ---')
        display(df.sample(5))

Overwriting pandas_utils.py


In [3]:
from pandas_utils import describe

## Read one2one h5ad

In [72]:
ls -lha /home/olga/data_sm/immune-evolution/h5ads/

total 1.7G
drwxrwxr-x 6 phoenix czb 4.0K Feb  3 09:47 ./
drwxrwxr-x 8 olga    czb 8.0K Mar 16 10:36 ../
drwxrwxr-x 2 phoenix czb 4.0K Feb  3 09:48 bat/
-rw-r--r-- 1 phoenix czb 496M Oct  8 17:45 bat_subset_w_shared_one2one_orthologs_human_lemur_mouse_bat.h5ad
drwxr-xr-x 2 phoenix czb 4.0K Oct  8 17:51 human_lemur_mouse/
drwxr-xr-x 2 olga    czb 4.0K Nov  5 07:54 human-lemur-mouse-bat/
-rw-r--r-- 1 phoenix czb 470M Oct  8 17:45 human_lemur_mouse_subset_w_shared_one2one_orthologs_human_lemur_mouse_bat.h5ad
drwxr-xr-x 2 olga    czb 4.0K Jan 11 15:00 human-mouse/
-rw-r--r-- 1 olga    czb 689M Feb  3 09:46 human-mouse-bat__one2one_orthologs_raw.h5ad
-rw-r--r-- 1 olga    czb 546K Feb  3 13:27 human-mouse-bat__one2one_orthologs_var.csv


In [76]:
ad = sc.read('/home/olga/data_sm/immune-evolution/h5ads/human-mouse-bat__one2one_orthologs_raw.h5ad')
ad.obs.tissue.value_counts()

Lung    63575
Name: tissue, dtype: int64

In [88]:
ad.obs.groupby(['species', 'individual'], observed=True).size()

species  individual
Human    TSP1          11628
         TSP2          19489
Mouse    18-F-50         320
         18-F-51        1138
         18-M-52        1573
         18-M-53        1256
         21-F-54        1312
         21-F-55        1526
         30-M-2         7728
         30-M-3         1565
         30-M-4          920
         30-M-5          496
         1-M-62         1043
         1-M-63         1384
         3-F-56          348
         3-F-57          979
         3-M-5/6         578
         3-M-7/8        1636
nan      nan            8656
dtype: int64

In [86]:
ad.obs.head()

,original_cell_id,individual,tissue,subtissue,cell_barcode,channel,sex,n_counts,n_genes,cell_ontology_class,...,clustertype,cluster,celltype,celltype_tiss,clustercellnumber,newcelltype,clustertonumber,tissue_lower,sex_abbrev,cell_id
TSP1_lung_2__AAACCCACAACTCGTA-0,AAACCCACAACTCGTA_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCACAACTCGTA,TSP1_lung_2,female,31791.0,1470.0,basophil,...,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan
TSP1_lung_2__AAACCCAGTCCGACGT-0,AAACCCAGTCCGACGT_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCAGTCCGACGT,TSP1_lung_2,female,19200.0,3756.0,macrophage,...,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan
TSP1_lung_2__AAACCCAGTTCAGTAC-0,AAACCCAGTTCAGTAC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCAGTTCAGTAC,TSP1_lung_2,female,20952.0,3885.0,macrophage,...,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan
TSP1_lung_2__AAACGAAGTATCGATC-0,AAACGAAGTATCGATC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACGAAGTATCGATC,TSP1_lung_2,female,20300.0,4043.0,macrophage,...,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan
TSP1_lung_2__AAACGAATCATTCATC-0,AAACGAATCATTCATC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACGAATCATTCATC,TSP1_lung_2,female,20878.0,4284.0,macrophage,...,nan,nan,nan,nan,NaN,nan,NaN,nan,nan,nan


In [78]:
ad.obs.groupby(['species', 'tissue']).size()

species  tissue
Human    Lung      31117
Mouse    Lung      23802
nan      Lung       8656
dtype: int64

In [73]:
ls -lha /home/olga/data_sm/immune-evolution/h5ads/human-mouse/

total 15G
drwxr-xr-x 2 olga    czb 4.0K Jan 11 15:00 ./
drwxrwxr-x 6 phoenix czb 4.0K Feb  3 09:47 ../
-rw-r--r-- 1 olga    czb 444M Jan 11 15:05 human-mouse-one2one-orthologs--bladder.h5ad
-rw-r--r-- 1 olga    czb 5.1G Jan 11 15:04 human-mouse-one2one-orthologs.h5ad
-rw-r--r-- 1 olga    czb 791M Jan 11 15:04 human-mouse-one2one-orthologs--lung.h5ad
-rw-r--r-- 1 olga    czb 209M Jan 11 15:04 human-mouse-one2one-orthologs--muscle.h5ad
-rw-r--r-- 1 olga    czb 3.1G Jan 11 14:58 tabula-muris-senis--human-mouse-one2one-ortholog-subset.h5ad
-rw-r--r-- 1 olga    czb 4.9G Jan 11 14:50 tabula-sapiens--human-mouse-one2one-ortholog-subset.h5ad


In [85]:
! echo "This is Tabula Sapiens + Tabula Muris Senis" >> /home/olga/data_sm/immune-evolution/h5ads/human-mouse/README.md

In [80]:
human_mouse = sc.read('/home/olga/data_sm/immune-evolution/h5ads/human-mouse/human-mouse-one2one-orthologs--lung.h5ad')
human_mouse

AnnData object with n_obs × n_vars = 54919 × 14245
    obs: 'original_cell_id', 'individual', 'tissue', 'subtissue', 'cell_barcode', 'channel', 'sex', 'n_counts', 'n_genes', 'cell_ontology_class', 'species', 'species_latin', 'narrow_group', 'broad_group', 'compartment_group', 'age', 'cell_ontology_id', 'free_annotation', 'sequencing_run', 'species_batch'
    var: 'ensembl_id-human', 'len-human', 'mouse_gene_symbol-mouse', 'n_cells-mouse'

In [82]:
human_mouse.obs.species_batch.value_counts()

human    31117
mouse    23802
Name: species_batch, dtype: int64

In [83]:
human_mouse.obs.head()

,original_cell_id,individual,tissue,subtissue,cell_barcode,channel,sex,n_counts,n_genes,cell_ontology_class,species,species_latin,narrow_group,broad_group,compartment_group,age,cell_ontology_id,free_annotation,sequencing_run,species_batch
cell_id,,,,,,,,,,,,,,,,,,,,
TSP1_lung_2__AAACCCACAACTCGTA,AAACCCACAACTCGTA_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCACAACTCGTA,TSP1_lung_2,female,31791.0,1470.0,basophil,Human,Homo sapiens,nan,nan,nan,nan,nan,nan,nan,human
TSP1_lung_2__AAACCCAGTCCGACGT,AAACCCAGTCCGACGT_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCAGTCCGACGT,TSP1_lung_2,female,19200.0,3756.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_lung_2__AAACCCAGTTCAGTAC,AAACCCAGTTCAGTAC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACCCAGTTCAGTAC,TSP1_lung_2,female,20952.0,3885.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_lung_2__AAACGAAGTATCGATC,AAACGAAGTATCGATC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACGAAGTATCGATC,TSP1_lung_2,female,20300.0,4043.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human
TSP1_lung_2__AAACGAATCATTCATC,AAACGAATCATTCATC_TSP1_lung_2-5-human,TSP1,Lung,nan,AAACGAATCATTCATC,TSP1_lung_2,female,20878.0,4284.0,macrophage,Human,Homo sapiens,macrophage,macrophage,myeloid,nan,nan,nan,nan,human


In [4]:
adata = sc.read(
#     "/home/phoenix/data_sm/code/immune-evolution/h5ads/human-lemur-mouse-bat__lung_only_copy.h5ad"
    "/home/olga/data_sm/immune-evolution/h5ads/human-lemur-mouse-bat/human-lemur-mouse-bat__lung_only.h5ad"
)
adata.obs = adata.obs.reset_index().set_index('cell_id')
print(adata)
adata.obs.head()

AnnData object with n_obs × n_vars = 126745 × 10560
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'channel_cleaned', 'batch', 'n_genes', 'n_counts', 'species_batch_v2', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'common_individual_id'
    var: 'bat__gene_name', 'mouse_lemur__gene_name-bat', 'mouse__gene_name-bat', 'mouse_lemur__gene_name_x-hlm', 'mouse__gene_name_x-hlm', 'gene_ids-lemur-hlm', 'n_cells-mouse-hlm', 'mouse_lemur__gene_name_y-hlm', 'mouse__gene_name_y-hlm'


,index,age,cell_barcode,cell_ontology_class,cell_ontology_id,channel,free_annotation,individual,sample,sequencing_run,...,channel_cleaned,batch,n_genes,n_counts,species_batch_v2,compartment_broad,compartment_broad_narrow,compartment_species,compartment_narrow_species,common_individual_id
cell_id,,,,,,,,,,,,,,,,,,,,,
P2_1__AAACCTGAGAAACCAT,P2_1_AAACCTGAGAAACCAT-human-hlm,46y,AAACCTGAGAAACCAT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AAATGCCAGATGAGAG,P2_1_AAATGCCAGATGAGAG-human-hlm,46y,AAATGCCAGATGAGAG,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACACGTTCGATCCCT,P2_1_AACACGTTCGATCCCT-human-hlm,46y,AACACGTTCGATCCCT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACACGTTCGCACTCT,P2_1_AACACGTTCGCACTCT-human-hlm,46y,AACACGTTCGCACTCT,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2
P2_1__AACCATGCAGCTCGCA,P2_1_AACCATGCAGCTCGCA-human-hlm,46y,AACCATGCAGCTCGCA,capillary endothelial cell,nan,P2_1,Capillary Aerocyte,patient_2,distal 2,180607_A00111_0155_BHFCWYDMXX,...,P2_1,NaN,NaN,NaN,hlm,endothelial: Capillary,endothelial: Capillary: Capillary Aerocyte,endothelial: human,endothelial: Capillary Aerocyte: human,human_patient_2


In [5]:
adata.obs.species.value_counts()

Human          57353
Mouse lemur    36934
Mouse          23802
Bat             8656
Name: species, dtype: int64

In [6]:
adata.obs.query('species == "Mouse"').groupby(["age", "channel"], observed=True).size()

age  channel           
18m  MACA_18m_F_LUNG_50     320
     MACA_18m_F_LUNG_51    1138
     MACA_18m_M_LUNG_52    1573
     MACA_18m_M_LUNG_53    1256
21m  MACA_21m_F_LUNG_54    1312
     MACA_21m_F_LUNG_55    1526
30m  10X_P1_6              7728
     10X_P1_14             1565
     10X_P2_5               920
     10X_P3_0               496
1m   10X_P5_13             1043
     10X_P5_14             1384
3m   10X_P7_8               348
     10X_P7_9               979
     10X_P8_12              578
     10X_P8_13             1636
dtype: int64

## Set shared celltypes

In [7]:
SHARED_CELLTYPES = ['Alveolar Epithelial Type 2',
 'B cell',
 'Capillary',
 'Dendritic',
 'Fibroblast',
 'Macrophage',
 'Monocyte',
 'Natural Killer T cell',
 'Smooth Muscle and Myofibroblast',
 'T cell']
len(SHARED_CELLTYPES)

10

In [8]:
adata_shared = adata[adata.obs.broad_group.isin(SHARED_CELLTYPES)]
adata_shared

/home/olga/miniconda3/envs/immune-evolution/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


View of AnnData object with n_obs × n_vars = 89795 × 10560
    obs: 'index', 'age', 'cell_barcode', 'cell_ontology_class', 'cell_ontology_id', 'channel', 'free_annotation', 'individual', 'sample', 'sequencing_run', 'sex', 'species', 'species_batch', 'species_latin', 'tissue', 'narrow_group', 'broad_group', 'compartment_group', 'compartment_narrow', 'channel_cleaned', 'batch', 'n_genes', 'n_counts', 'species_batch_v2', 'compartment_broad', 'compartment_broad_narrow', 'compartment_species', 'compartment_narrow_species', 'common_individual_id'
    var: 'bat__gene_name', 'mouse_lemur__gene_name-bat', 'mouse__gene_name-bat', 'mouse_lemur__gene_name_x-hlm', 'mouse__gene_name_x-hlm', 'gene_ids-lemur-hlm', 'n_cells-mouse-hlm', 'mouse_lemur__gene_name_y-hlm', 'mouse__gene_name_y-hlm'

In [9]:
CELL_IDS_TO_USE = set(adata_shared.obs.index)
len(CELL_IDS_TO_USE)

89795

In [10]:
# adata_shared.obs.query('species == "Mouse"').groupby(
#     ["channel", "broad_group"], observed=True
# ).size()

In [11]:
adata_shared.obs.query('species == "Mouse"').groupby(
    ['age', "channel"],
    observed=True,
).broad_group.nunique()

age  channel           
18m  MACA_18m_F_LUNG_50     9
     MACA_18m_F_LUNG_51    10
     MACA_18m_M_LUNG_52    10
     MACA_18m_M_LUNG_53    10
21m  MACA_21m_F_LUNG_54    10
     MACA_21m_F_LUNG_55    10
30m  10X_P1_6               8
     10X_P1_14              9
     10X_P2_5               9
     10X_P3_0               9
1m   10X_P5_13             10
     10X_P5_14              9
3m   10X_P7_8              10
     10X_P7_9              10
     10X_P8_12              8
     10X_P8_13              8
Name: broad_group, dtype: int64

## Set signature outdir

In [13]:
ls -lha $sig_outdir_base

total 11K
drwxr-xr-x  6 olga czb 4.0K Feb 28 09:33 ./
drwxrwxr-x 81 olga czb 8.0K Feb 28 09:02 ../
drwxr-xr-x  4 olga czb 4.0K Feb 28 09:53 0--mouse2mouse/
drwxr-xr-x  4 olga czb 4.0K Feb 28 09:49 1--train-mouse/
drwxr-xr-x  2 olga czb 4.0K Feb 28 09:17 mouse/
drwxr-xr-x  3 olga czb 4.0K Feb 27 09:59 mouse2mouse/


In [14]:
rm -rf $sig_outdir_base/mouse/single-cell

## sketch ids of interest

In [15]:
alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))
sketch_ids = [sig_utils.make_sketch_id(alpha=alpha, ksize=ksize, style='scaled', value=10) for alpha, ksize in alpha_ksizes]
sketch_ids

['alphabet-DNA__ksize-21__scaled-10',
 'alphabet-protein__ksize-30__scaled-10',
 'alphabet-dayhoff__ksize-51__scaled-10']

In [16]:
nucleotide_sketch_ids = sketch_ids[:1]
nucleotide_sketch_ids

['alphabet-DNA__ksize-21__scaled-10']

In [17]:
peptide_sketch_ids = sketch_ids[1:]
peptide_sketch_ids

['alphabet-protein__ksize-30__scaled-10',
 'alphabet-dayhoff__ksize-51__scaled-10']

In [45]:
import sig_reorg

# Human

## Single cell fastas

In [47]:
! ls -lha $sig_outdir_base

total 11K
drwxr-xr-x  6 olga czb 4.0K Feb 28 09:33 .
drwxrwxr-x 81 olga czb 8.0K Feb 28 09:02 ..
drwxr-xr-x  3 olga czb 4.0K Feb 28 09:33 0--mouse2mouse
drwxr-xr-x  3 olga czb 4.0K Feb 28 09:29 1--train-mouse
drwxr-xr-x  2 olga czb 4.0K Feb 28 09:17 mouse
drwxr-xr-x  3 olga czb 4.0K Feb 27 09:59 mouse2mouse


In [28]:
human_kmermaid_base = "/home/olga/data_sm/tabula-microcebus/analyses/kmermaid/tenx-tgz--hlca-v2--normal-lung-remove-riborna"

In [41]:
sig_outdir_base = '/home/olga/data_sm/immune-evolution/kmer-signatures'

human_test_dir = os.path.join(sig_outdir_base, "2--test-human")

human_test_0_fastas = os.path.join(human_test_dir, "0--fastas")
human_test_1_single_cells = os.path.join(human_test_dir, "1--single-cell-sigs")
human_fasta_input_dir = os.path.join(human_kmermaid_base, "translate")

results_dir = os.path.join(human_test_dir, "2--mouse-search-results")
dirs = (human_test_0_fastas, human_test_1_single_cells, results_dir)

for outdir in dirs:
    if not os.path.exists(outdir):
        os.makedirs(outdir)


## Copy fastas

In [65]:
sig_reorg.copy_fastas(
    human_test_0_fastas,
    human_fasta_input_dir,
    select_cell_ids=CELL_IDS_TO_USE,
    dryrun=False,
)

0it [00:00, ?it/s]

Copying peptides


54305it [03:21, 269.80it/s]
0it [00:00, ?it/s]

Copying nucleotides


54305it [03:21, 269.13it/s]


## Single cell signatures

In [30]:

human_peptide_sketch_dir = os.path.join(
    human_kmermaid_base,
    "sketches_peptide_handmade_merged_remove_ribosomal_dissociation_allcells",
)

human_nucleotide_sketch_dir = os.path.join(
    human_kmermaid_base, "sketches_nucleotide_handmade_merged_remove_ribosomal_dissociation_allcells"
)

In [31]:
sig_reorg.copy_nucleotide_peptide_sketches(
    peptide_sketch_dir=human_peptide_sketch_dir,
    nucleotide_sketch_dir=human_nucleotide_sketch_dir,
    pre_sketch_id_outdir=human_test_1_single_cells,
    select_cell_ids=CELL_IDS_TO_USE,
    dryrun=False
)

219it [00:00, 2189.94it/s]

Copying alphabet-DNA__ksize-21__scaled-10


14153it [03:32, 66.47it/s] 
8it [00:00, 63.92it/s]

Copying alphabet-protein__ksize-30__scaled-10


20059it [01:27, 230.41it/s]
22it [00:00, 212.30it/s]

Copying alphabet-dayhoff__ksize-51__scaled-10


20059it [01:26, 231.48it/s]


# Create queries

In [45]:
human_test_dir

'/home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human'

## celltypes with removed common k-mers

In [62]:
moltypes = "DNA", "protein", "dayhoff"

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

# mouse celltype database directory
train_4_celltype_sbts = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/4--celltype-sbt-dbts"
train_3_merged_celltype_remove_common = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers"
train_2_merged_celltype = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--merged-celltype-sigs"

# Try both with removed common hashes and original
celltype_dirs = dict(
    remove_common=train_3_merged_celltype_remove_common,
    raw_merged=train_2_merged_celltype,
)

txts = []
for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        human_test_dir, f"3-{i}--mouse-search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results,
            query_sigs_dir=human_test_1_single_cells,
            sbt_base_dir=celltype_dir,
            k_sizes=[ksize],
            scaled_sizes=[
                10,
            ],
            sbt_template_basename=None,
            query_sig_files=False,
            containment=False,
            moltype=alpha,
            threshold=1e-100,
            n_jobs=96,
            force=True,
            num_results=None,
            add_ksize_to_sig_dir=False,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = " ".join(txts)
catted_txt = os.path.join(results_dir, "sourmash_search_commands_concatenated.txt")
! cat $txts_to_cat > $catted_txt

print(f"---\nparallel --progress --eta --jobs 96 < {catted_txt}")

parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--remove_common/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--remove_common/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--remove_common/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--raw_merged/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--raw_merged/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/

## Search mouse celltypes with k-mers shared by 10% of cells

In [68]:
moltypes = "DNA", "protein", "dayhoff"

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

# mouse celltype database directory
train_4_celltype_sbts = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/4--celltype-sbt-dbts"
train_3_sigs_min_10_percent = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--merged-celltype-sigs--min-kmer-count--10-percent"
train_3_sigs_min_5_percent = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--merged-celltype-sigs--min-kmer-count--5-percent"
# train_2_merged_celltype = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/2--merged-celltype-sigs"

# Try both with removed common hashes and original
celltype_dirs = {
    'min-5-percent-shared-kmers': train_3_sigs_min_5_percent,
    'min-10-percent-shared-kmers': train_3_sigs_min_10_percent,
}

txts = []
for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        human_test_dir, f"3-{i}--mouse-search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results,
            query_sigs_dir=human_test_1_single_cells,
            sbt_base_dir=celltype_dir,
            k_sizes=[ksize],
            scaled_sizes=[
                10,
            ],
            sbt_template_basename=None,
            query_sig_files=False,
            containment=False,
            moltype=alpha,
            threshold=1e-100,
            n_jobs=96,
            force=True,
            num_results=None,
            add_ksize_to_sig_dir=False,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = " ".join(txts)
catted_txt = os.path.join(results_dir, "sourmash_search_commands_concatenated.txt")
! cat $txts_to_cat > $catted_txt

print(f"---\nparallel --progress --eta --jobs 96 < {catted_txt}")

parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--min-10-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--min-10-percent-shared-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --job

## Search mouse celltypes with k-mers shared by 5, 10% of cells, but remove k-mers shared by 80% of celltypes

In [70]:
moltypes = "DNA", "protein", "dayhoff"

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

# mouse celltype database directory
train_4_celltype_sbts = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/4--celltype-sbt-dbts"
train_3_sigs_min_5_percent_remove_common = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent"
train_3_sigs_min_10_percent_remove_common = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent"

# Try both with removed common hashes and original
celltype_dirs = {
    'min-5-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_5_percent_remove_common,
    'min-10-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_10_percent_remove_common,
}

txts = []
for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        human_test_dir, f"3-{i}--mouse-search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results,
            query_sigs_dir=human_test_1_single_cells,
            sbt_base_dir=celltype_dir,
            k_sizes=[ksize],
            scaled_sizes=[
                10,
            ],
            sbt_template_basename=None,
            query_sig_files=False,
            containment=False,
            moltype=alpha,
            threshold=1e-100,
            n_jobs=96,
            force=True,
            num_results=None,
            add_ksize_to_sig_dir=False,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = " ".join(txts)
catted_txt = os.path.join(results_dir, "sourmash_search_commands_concatenated.txt")
! cat $txts_to_cat > $catted_txt

print(f"---\nparallel --progress --eta --jobs 96 < {catted_txt}")

parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers-remove-common-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers-remove-common-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-search-results--min-5-percent-shared-kmers-remove-common-kmers/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--min-10-percent-shared-kmers-remove-common-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-search-results--min-10-percent-s

## CONTAINMENT - Search mouse celltypes with k-mers shared by 5, 10% of cells, but remove k-mers shared by 80% of celltypes

In [71]:
moltypes = "DNA", "protein", "dayhoff"

alpha_ksizes = (("DNA", 21), ("protein", 30), ("dayhoff", 51))

# mouse celltype database directory
train_4_celltype_sbts = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/4--celltype-sbt-dbts"
train_3_sigs_min_5_percent_remove_common = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--5-percent"
train_3_sigs_min_10_percent_remove_common = "/home/olga/data_sm/immune-evolution/kmer-signatures/1--train-mouse/3--merged-celltype-remove-common-kmers--min-kmer-count--10-percent"

# Try both with removed common hashes and original
celltype_dirs = {
        'min-5-percent-shared-kmers': train_3_sigs_min_5_percent,
    'min-10-percent-shared-kmers': train_3_sigs_min_10_percent,
    'min-5-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_5_percent_remove_common,
    'min-10-percent-shared-kmers-remove-common-kmers': train_3_sigs_min_10_percent_remove_common,
}

txts = []
for i, (name, celltype_dir) in enumerate(celltype_dirs.items()):
    this_celltype_dir_results = os.path.join(
        human_test_dir, f"3-{i}--mouse-containment-search-results--{name}"
    )
    if not os.path.exists(this_celltype_dir_results):
        os.makedirs(this_celltype_dir_results)
    for alpha, ksize in alpha_ksizes:
        txt = csc_utils.make_sourmash_search_commands(
            this_celltype_dir_results,
            query_sigs_dir=human_test_1_single_cells,
            sbt_base_dir=celltype_dir,
            k_sizes=[ksize],
            scaled_sizes=[
                10,
            ],
            sbt_template_basename=None,
            query_sig_files=False,
            containment=True,
            moltype=alpha,
            threshold=1e-100,
            n_jobs=96,
            force=True,
            num_results=None,
            add_ksize_to_sig_dir=False,
            traverse_directory=True,
        )
        txts.append(txt)
txts_to_cat = " ".join(txts)
catted_txt = os.path.join(results_dir, "sourmash_containment_search_commands_concatenated.txt")
! cat $txts_to_cat > $catted_txt

print(f"---\nparallel --progress --eta --jobs 96 < {catted_txt}")

parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_protein.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-0--mouse-containment-search-results--min-5-percent-shared-kmers/sourmash_search_commands_dayhoff.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-containment-search-results--min-10-percent-shared-kmers/sourmash_search_commands_DNA.txt
parallel --progress --eta --jobs 96 < /home/olga/data_sm/immune-evolution/kmer-signatures/2--test-human/3-1--mouse-containment-search-results--min-10-percent-shared-kmers/sourmash